In [ ]:
#web

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time 
import sqlite3
from datetime import date
import datetime

In [2]:
# get all content from 2 pages

url = "https://www.gulliver.co.il/Content/SalePage.aspx?pageId=264&module=Flights"
r = requests.get(url)
c = r.content
soup=BeautifulSoup(c,"html.parser")
all = soup.find_all("ul")

url2 = 'https://www.gulliver.co.il/Content/SalePage.aspx?pageId=99&module=Flights'
r2 = requests.get(url2)
c2 = r2.content
soup2=BeautifulSoup(c2,"html.parser")
all2 = soup2.find_all("ul")  




In [3]:
# get relevan data from mivzaim page into array

sale = []

for i in range(len(soup.find_all("strong",{'style':'font-size:1em;'}))):
    m={}
    m['destination'] = soup.find_all("strong",{'style':'font-size:1em;'})[i].text.replace('\t', '')
    m['company'] = soup.find_all("span", {'class':'LastDeals_hotel'})[i].text.replace('\t', '')
    m['vacationDate'] = soup.find_all("span", {'class':'LastDeals_Date'})[i].text.replace('\t', '')
    m['PriceInDollars'] = int(soup.find_all("strong", {'class':'LastDeals_price'})[i].text.replace('$',''))
    m['offerDate'] = date.today()
    sale.append(m)

In [4]:
# get relevan data from last minute page into array

last = []

for i in range(len(all2)):
    m={}
    m['destination'] = soup2.find_all("strong",{'style':'font-size:1em;'})[i].text.replace('\t', '').split()[0]
    m['company'] = 'Last Minute'
    m['vacationDate'] = soup2.find_all("span", {'class':'LastDeals_Details'})[i].text.replace('מחיר לאדםתאריכי טיסה','') 
    m['PriceInDollars'] = int(soup2.find_all("strong", {'class':'LastDeals_price'})[i].text.replace('$','').replace('€',''))
    m['offerDate'] = date.today()
    last.append(m)

In [5]:
# insert arrays into dataframes

df=pd.DataFrame(sale)
cols = df.columns.tolist()
cols = [cols[3] , cols[2] , cols[1],cols[4],cols[0]]
df = df[cols] 

df2 = pd.DataFrame(last)
df2 = df2[cols]

#merge the 2 dfs
merged = df2.append(df,ignore_index=True)

In [36]:
# sql 


def create():
    conn=sqlite3.connect("flights.db")
    cur=conn.cursor()
    cur.execute("CREATE TABLE IF NOT EXISTS flights (id INTEGER PRIMARY KEY,offerDate INTEGER,destination TEXT,company TEXT,vacationDate TEXT,PriceInDollars INTEGER)")
    conn.commit()
    conn.close()
          
    
def gettable():
    conn = sqlite3.connect("flights.db")
    table = pd.read_sql_query("SELECT * FROM flightData", conn)
    return table
    conn.close()        
    

def drop_table():
    conn = sqlite3.connect("flights.db")
    cur = conn.cursor()
    cur.execute("DROP TABLE flightData")
    conn.commit()
    conn.close()
    
def insert(merged) :  
    conn = sqlite3.connect("flights.db")   
    df = merged
    df.to_sql("flightData", conn, if_exists='append')
    conn.commit()
    conn.close()

    
def findLastDate():
    conn=sqlite3.connect("flights.db") 
    cur = conn.cursor()
    lastDate = cur.execute("SELECT distinct max(offerDate) FROM flightData").fetchone()[0]
    conn.close()
    return lastDate


In [8]:
# insert to db if offer date if offerdate is not exist

if  (datetime.datetime.strptime(findLastDate(),"%Y-%m-%d").date()) < max(merged.offerDate) :
    insert(merged)
    print("inserted")
else :
    print ("Already in database")
    print ("DB last date :" , findLastDate())
    print ("current date :" ,max(merged.offerDate))


Already in database
DB last date : 2018-07-07
current date : 2018-07-07


In [44]:

def highlight_price(price,lowerPrice=100,mediumPrice=180):
    
    if price < lowerPrice :
        color = 'grey' 
        
    elif price < mediumPrice :
        color = 'green'
        
    else :
        color = None
        
    return 'background-color: %s' % color


def highlight_war(s,destination='ורשה'):
    if s == destination :
        color = 'yellow'
    else : color = None
    return 'background-color: %s' % color

    
s=df.style.applymap(highlight_price, subset=pd.IndexSlice[:, ['PriceInDollars']]).applymap(highlight_war, subset=pd.IndexSlice[:, ['destination']])

In [43]:
s

,index,offerDate,destination,company,vacationDate,PriceInDollars
0,0,2018-06-24,אנטליה,Last Minute,28.06 - 24.06,132
1,1,2018-06-24,רודוס,Last Minute,02.07 - 25.06,161
2,2,2018-06-24,כרתים,Last Minute,01.07 - 25.06,161
3,3,2018-06-24,לרנקה,Last Minute,04.07 - 25.06,164
4,4,2018-06-24,קטניה,Last Minute,04.07 - 27.06,151
5,5,2018-06-24,ורנה,Last Minute,02.07 - 25.06,179
6,6,2018-06-24,ורונה,Last Minute,04.07 - 27.06,179
7,7,2018-06-24,קוס,Last Minute,29.06 - 25.06,180
8,8,2018-06-24,בלגרד,Last Minute,29.06 - 25.06,187
9,9,2018-06-24,בוקרשט,Last Minute,10.07 - 25.06,191


In [37]:
df = gettable()

def get_des (destination):
    return df[df['destination'] == destination]
    
def get_under_price(price):
    return df[df['PriceInDollars'] < price]

def get_last () :
        return df[df['offerDate'] == max(df['offerDate'])]

def get_destination():
    return df['destination'].unique()


In [46]:
destination = 'ורשה'
get_des(destination).sort_values('PriceInDollars')


,index,offerDate,destination,company,vacationDate,PriceInDollars
35,35,2018-06-24,ורשה,LOT,ליציאות בחודשים נובמבר - מרץ,178
77,77,2018-06-26,ורשה,LOT,ליציאות בחודשים נובמבר - מרץ,178
119,119,2018-06-27,ורשה,LOT,ליציאות בחודשים נובמבר - מרץ,178
159,159,2018-02-07,ורשה,LOT,ליציאות בחודשים נובמבר - מרץ,178
203,37,2018-07-06,ורשה,LOT,ליציאות בחודשים נובמבר - מרץ,178
247,37,2018-07-07,ורשה,LOT,ליציאות בחודשים נובמבר - מרץ,178


In [45]:
get_under_price(150).sort_values('PriceInDollars')

,index,offerDate,destination,company,vacationDate,PriceInDollars
30,30,2018-06-24,פאפוס,RYNAIR,30.9-9.10,52
72,72,2018-06-26,פאפוס,RYNAIR,30.9-9.10,52
244,34,2018-07-07,פאפוס,RYNAIR,28.9-9.10,54
200,34,2018-07-06,פאפוס,RYNAIR,28.9-9.10,54
31,31,2018-06-24,פאפוס,RYNAIR,1-11.10,59
114,114,2018-06-27,פאפוס,RYNAIR,30.9-9.10,59
156,156,2018-02-07,פאפוס,RYNAIR,1-11.10,61
115,115,2018-06-27,פאפוס,RYNAIR,1-11.10,61
73,73,2018-06-26,פאפוס,RYNAIR,1-11.10,61
157,157,2018-02-07,בורגס,RYNAIR,10-14.10,69


In [47]:
get_last().sort_values('PriceInDollars')

,index,offerDate,destination,company,vacationDate,PriceInDollars
244,34,2018-07-07,פאפוס,RYNAIR,28.9-9.10,54
245,35,2018-07-07,בורגס,RYNAIR,10-14.10,74
210,0,2018-07-07,רודוס,Last Minute,11.07 - 07.07,104
211,1,2018-07-07,ורנה,Last Minute,12.07 - 08.07,108
246,36,2018-07-07,אתונה,AGEAN,ליציאות בחודשים נובמבר - מרץ,111
212,2,2018-07-07,זקינטוס,Last Minute,11.07 - 07.07,113
213,3,2018-07-07,בורגס,Last Minute,12.07 - 09.07,127
214,4,2018-07-07,לרנקה,Last Minute,13.07 - 10.07,137
215,5,2018-07-07,כרתים,Last Minute,15.07 - 09.07,142
216,6,2018-07-07,מיקונוס,Last Minute,13.07 - 09.07,142
